In [1]:
USE {
    dependencies(
        "commons-io:commons-io:2.20.0",
    )
}

In [2]:
%use adventOfCode, dataframe

In [3]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 6)
aoc.viewPartOne()

--- Day 6: Trash Compactor --- 
 After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute! 
 A brief fall later, you find yourself in a garbage smasher . Unfortunately, the door's been magnetically sealed. 
 As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her math homework . 
 Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of problems ; each problem has a group of numbers that need to be either added ( + ) or multiplied ( * ) together. 
 However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example: 
 123 328 51 64 
 45 64 387 23 
 6 98 215 314
* + * + 
 
 Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored. 
 So, this worksheet contains four problems: 
 
 123 * 45 * 6 = 33210 
 328 + 64 + 98 = 490 
 51 * 387 * 215 = 4243455 
 64 + 23 + 314 = 401 
 
 To check their work, cephalopod students are given the grand total of adding together all of the answers to the individual problems. In this worksheet, the grand total is 33210 + 490 + 4243455 + 401 = 4277556 . 
 Of course, the actual worksheet is much wider. You'll need to make sure to unroll it completely so that you can read the problems clearly. 
 Solve the problems on the math worksheet. What is the grand total found by adding together all of the answers to the individual problems?

In [58]:
val exampleInput = """123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +"""

In [78]:
enum class Operation(val char: Char, val apply: (Long, Long) -> Long) {
    TIMES('*', Long::times),
    PLUS('+', Long::plus),
    ;

    companion object {
        fun of(char: Char) = entries.find { it.char == char }!!
    }
}

data class Problem(val numbers: List<Long>, val operation: Operation) {
    fun solve(): Long = numbers.reduce(operation.apply)
}

fun String.parse(): List<Problem> {
    val lines = lines()
    val width = lines.maxOf { it.length }
    var operationLine = lines.last().padEnd(width) // important! some lines end early

    // calculate the operation and the width of their column
    val operationColWidths: List<Pair<Char, Int>> =
        operationLine.split('*', '+').zipWithNext().map { (operation, space) ->
            val operation = operationLine.take(operation.length + 1)
            operationLine = operationLine.drop(operation.length)

            operation.trim().single() to space.length + 1
        }

    val numberLines: MutableList<String> = lines.dropLast(1).toMutableList()
    return operationColWidths.map { (operation, width) ->
        Problem(
            numbers = buildList {
                for (j in numberLines.indices) {
                    this += numberLines[j].take(width).trim().toLong()
                    numberLines[j] = numberLines[j].drop(width)
                }
            },
            operation = Operation.of(operation),
        )
    }
}

In [79]:
exampleInput.parse()

[Problem(numbers=[123, 45, 6], operation=TIMES), Problem(numbers=[328, 64, 98], operation=PLUS), Problem(numbers=[51, 387, 215], operation=TIMES), Problem(numbers=[64, 23, 314], operation=PLUS)]

In [80]:
exampleInput.parse().sumOf { it.solve() }

4277556

In [82]:
val input = aoc.input().parse()
aoc.submitPartOne(
    input.sumOf { it.solve().toLong() }
)

Your answer: 4805473544166. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [83]:
aoc.viewPartTwo()

--- Part Two --- 
 The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math. 
 Cephalopod math is written right-to-left in columns . Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.) 
 Here's the example worksheet again: 
 123 328 51 64 
 45 64 387 23 
 6 98 215 314
* + * + 
 
 Reading the problems right-to-left one column at a time, the problems are now quite different: 
 
 The rightmost problem is 4 + 431 + 623 = 1058 
 The second problem from the right is 175 * 581 * 32 = 3253600 
 The third problem from the right is 8 + 248 + 369 = 625 
 Finally, the leftmost problem is 356 * 24 * 1 = 8544 
 
 Now, the grand total is 1058 + 3253600 + 625 + 8544 = 3263827 . 
 Solve the problems on the math worksheet again. What is the grand total found by adding together all of the answers to the individual problems?

In [85]:
exampleInput

123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +

prototyping...

In [98]:
val lines = exampleInput.lines()
val width = lines.maxOf { it.length }
var operationLine = lines.last().padEnd(width) // important! some lines end early

// calculate the operation and the width of their column
val operationColWidths: List<Pair<Char, Int>> =
    operationLine.split('*', '+').zipWithNext().map { (operation, space) ->
        val operation = operationLine.take(operation.length + 1)
        operationLine = operationLine.drop(operation.length)

        operation.trim().single() to space.length + 1
    }

DISPLAY(operationColWidths)

val numberLines: MutableList<String> = lines.dropLast(1).toMutableList()
operationColWidths.map { (operation, width) ->
    val numbers = (0..width - 1).mapNotNull { i ->
        numberLines.indices.map { j ->
            numberLines[j].getOrElse(i) { ' ' }
        }.joinToString("")
            .trim()
            .toLongOrNull()
    }
    numberLines.indices.forEach { j -> numberLines[j] = numberLines[j].drop(width) }

    Problem(
        numbers = numbers,
        operation = Operation.of(operation),
    )
}

[(*, 4), (+, 4), (*, 4), (+, 3)]

[Problem(numbers=[1, 24, 356], operation=TIMES), Problem(numbers=[369, 248, 8], operation=PLUS), Problem(numbers=[32, 581, 175], operation=TIMES), Problem(numbers=[623, 431, 4], operation=PLUS)]

In [100]:
fun String.parsePt2(): List<Problem> {
    val lines = this.lines()
    val width = lines.maxOf { it.length }
    var operationLine = lines.last().padEnd(width) // important! some lines end early

    // calculate the operation and the width of their column
    val operationColWidths: List<Pair<Char, Int>> =
        operationLine.split('*', '+').zipWithNext().map { (operation, space) ->
            val operation = operationLine.take(operation.length + 1)
            operationLine = operationLine.drop(operation.length)

            operation.trim().single() to space.length + 1
        }

    val numberLines: MutableList<String> = lines.dropLast(1).toMutableList()
    return operationColWidths.map { (operation, width) ->
        val numbers = (0..width - 1).mapNotNull { i ->
            numberLines.indices.map { j ->
                numberLines[j].getOrElse(i) { ' ' }
            }.joinToString("")
                .trim()
                .toLongOrNull()
        }
        numberLines.indices.forEach { j -> numberLines[j] = numberLines[j].drop(width) }

        Problem(
            numbers = numbers,
            operation = Operation.of(operation),
        )
    }
}

In [102]:
exampleInput.parsePt2().sumOf { it.solve() }

3263827

In [105]:
aoc.submitPartTwo(
    aoc.input().parsePt2().sumOf { it.solve() }
)

Your answer: 8907730960817. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 6! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .